In [4]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import functions as fun
sourcedir = '/Volumes/My Passport/cmip5/cmip5'

In [5]:
def Gmask(model):
    print(model)
    try:
        with xr.open_dataset(f'{sourcedir}/fx/sftgif/{model}/r0i0p0/sftgif_fx_{model}_historical_r0i0p0.nc') as ds:
            lon = ds['lon'].values
            lat = ds['lat'].values
            sft = ds['sftgif'].values
    except:
        with xr.open_dataset(f'{sourcedir}/fx/sftlf/{model}/r0i0p0/sftlf_fx_{model}_historical_r0i0p0.nc') as ds:
            lon = ds['lon'].values
            lat = ds['lat'].values
            sft = ds['sftlf'].values        
    lon[lon<0] = lon[lon<0]+360
    lons,lats = np.meshgrid(lon,lat)
    mask = np.ones(sft.shape)
    mask[lons<289] = 0
    mask[lons>350] = 0
    mask[lats<56] = 0
    mask[np.logical_and(lons<303,lats<72)] = 0
    mask[np.logical_and(lons>333,lats<66)] = 0
    mask[np.logical_and(lons<292,lats>80)] = 0
    mask[np.logical_and(lons<297,lats>82)] = 0
    mask = mask*sft/np.max(sft)

    with xr.open_dataset(f'{sourcedir}/fx/orog/{model}/r0i0p0/orog_fx_{model}_historical_r0i0p0.nc') as ds:
        orog = ds['orog'].values

    mask2 = np.ones(orog.shape)
    mask2[lons<290] = 0
    mask2[lons>340] = 0
    mask2[lats<60] = 0
    mask2[lats>85] = 0
    mask2[orog<1000] = 0.

    fig,ax = plt.subplots(1,2)
    ax[0].pcolor(lons,lats,mask,cmap=plt.get_cmap('Blues_r'))
    ax[1].pcolor(lons,lats,mask2,cmap=plt.get_cmap('Blues_r'))
    for Ax in ax:
        Ax.set_xlim([270,355])
        Ax.set_ylim([50,90])
    plt.savefig(f'figures/Gmask/{model}.png')
    plt.close()
    
    msk  = xr.DataArray(mask, dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    msk2 = xr.DataArray(mask2,dims=('lat','lon'),coords={'lat':lat,'lon':lon})    
    
    ds = xr.Dataset({'mask':msk,'mask2':msk2})
    ds.to_netcdf(f'data/Gmask/{model}.nc')
    ds.close()

In [6]:
mods = fun.models()
for model in mods:
    Gmask(model)

bcc-csm1-1
CanESM2
CCSM4
CNRM-CM5
CSIRO-Mk3-6-0
GFDL-CM3
GISS-E2-R
HadGEM2-ES
inmcm4
IPSL-CM5A-LR
MIROC-ESM
MIROC5
MPI-ESM-LR
MRI-CGCM3
NorESM1-M
